In [4]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy import linalg

In [ ]:
def DMD(X, Y, tol=1e-12, k=-1):
    "k - ako znas koliko modova zelis"
    #X - Nxm, Y - Nxm
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    #kada radis SVD - razlicite metode kada samo sigma ili kada u, sigma i v
        # kada trebas samo sigme bolje - bolje samo sigma
            #sigme - QR metoda, sve - divide and conquer
    V=np.conjugate(V.T)
    #print(Sigma)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    return Z, Lambda


In [ ]:
def DMD_exact(X, Y, tol=1e-12, k=-1):
    """X - Nxm, Y - Nxm; !!Exact DMD ne vraća normirane Z[:,i]"""
    m=X.shape[1]; N=X.shape[0]
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) #svd vraca V*
    V=np.conjugate(V.T)
    #print(Sigma)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    S_k = ((np.conjugate(U.T)@Y)@V)/Sigma
    Lambda, W = np.linalg.eig(S_k)
    Z = (Y@V/Sigma)@W/Lambda
    return Z, Lambda

In [ ]:
def GEDMD(X, Y, tol=1e-12, type: str=None, k=-1, weights=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D #provjeri rjesava li sustav ili odmah podijeli.
    if(weights is not None):
        X = X*weights; Y=Y*weights
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    B_k = Y @ (V/Sigma)
    S_k = np.conjugate(U.T)@B_k
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    if(type=="exact"):
        Z = B_k@W
    r = np.linalg.norm(B_k@W - np.multiply(Z, Lambda), axis=0)
    return Z, Lambda, r

In [ ]:
def GEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1];
    Q, R = np.linalg.qr(np.concatenate((X,Y), axis=1))
    Z, L, r = GEDMD(R[:,:m], R[:,m:], tol, type)
    Z = Q@Z
    return Z, L, r

In [ ]:
def GEDMDQ(S, tol=1e-12, type: str=None, k=-1, weights=None):
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=S.shape[1]-1;
    Q, R = np.linalg.qr(S)
    R_x = R[:(m+1),:m]; R_y = R[:(m+1), 1:(m+1)]
    Z, Lambda, r = GEDMD(R_x, R_y, tol, type, k, weights)
    Z = Q@Z
    return Z, Lambda, r, Q, R

In [ ]:
def DMD_added_snap(B, Q, R, tol=1e-12, type: str=None,k=-1, weights=None, ngram=5):
    m=R.shape[1]-1; N=Q.shape[0]; b=B.shape[1]
    if(B.shape[0]!=N):
        print ("new data doesn't have the same dimension as the old data")
        return
    #R_new= np.conjugate(Q.T)@B
    ##gram schmidt - reorthogonalization
    B_tilde = np.zeros((Q.shape[1],B.shape[1])) 
    R_new = np.copy(B)
    for i in range(ngram): #moram li jedan po jedan?
        dB = np.conjugate(Q.T)@R_new
        B_tilde += dB
        R_new -= Q@dB
    Q_1, R_1 = np.linalg.qr(R_new) 
    R_new=np.concatenate((B_tilde, R_1), axis=0)
    R = np.concatenate((R, np.zeros((b,m+1))), axis=0)
    R = np.concatenate((R, R_new), axis=1)
    Q = np.concatenate((Q, Q_1), axis=1)
    m=m+b
    R_x = R[:(m+1),:m]; R_y = R[:(m+1), 1:(m+1)]
    Z, L, r = GEDMD(R_x, R_y, tol, type, k, weights)
    Z=Q@Z
    return Z, L, r, Q, R


In [10]:
def householder_for_dmd(R, l):
    """R - matrix in trapezoidal form; 
    l - the amount of former dmd values discarded (also the number of sub-diagonals below the main one in R)"""
    N = R.shape[1]; N0 = R.shape[0]

    #l se moze iz oblika R procitat:
    #l=N0-N
    
    v_all=np.empty((l+1,N))

    #HH transf od R (trapezoidalna) 
    for i in range(N):
        element=R[i:(i+l+1),i]  
        v = element.copy().reshape(-1,1) #ako stavim v=element - pokazuju na istu memoriju; ne radi!!
        v[0] = v[0] + np.sign(element[0])*np.linalg.norm(element)
        R[i:(i+l+1),i:] = R[i:(i+l+1),i:] - 2/(v.T@v)*v@(v.T@R[i:(i+l+1),i:])
        v_all[:,i] = v.reshape(-1)

    ##Q - from HH matrices
    hh_current = np.eye(N0, N0)
    v_current=v_all[:,-1].reshape(-1,1)
    hh_current[-l-1:,-l-1:] = np.eye(l+1,l+1)-2/(v_current.T@v_current)*(v_current@v_current.T)

    hh=hh_current #ovdje @ I

    for i in range(N-2,-1,-1):
        v_current = v_all[:,i].reshape(-1,1)
        hh_current = np.eye(l+1,l+1)-2/(v_current.T@v_current)*(v_current@v_current.T)
        hh[i:i+l+1, :] = hh_current@hh[i:i+l+1, :]
        #print(hh)


    return hh, R, v_all

In [11]:
def DMD_discarding_snap(l, Q, R, tol=1e-12, type: str=None,k=-1, weights=None):
    m=R.shape[0]
    hh, R, _ = householder_for_dmd(R[:,l:], l)
    R_x = R[:(m-l),:(m-l-1)]; R_y = R[:(m-l), 1:(m-l)]
    Z, L, r = GEDMD(R_x, R_y, tol, type,k, weights)
    Q=(Q@hh)[:,:m-l]
    Z=Q@Z
    return Z,L,r,Q, R[:m-l,:], hh

In [ ]:
def DMD_alpha_for_reconstruction(X, Z, indices, L, weights=None):
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S
    Z = modes,
    indices = indices which we want to work with - from 1/r graph
    weights = np.array tip ili lista"""
    #treba li se formirati Vandermondeova matrica?
    m=X.shape[1]; l=indices.shape[0]
    Z = Z[:,indices]
    Q, R = np.linalg.qr(Z) # Q je tipa duljina_snapshota(N) x l, R je tipa lxl
    if(weights is None):
        weights=np.ones((m)).reshape(-1)
    weights=np.array(weights)
    pom=np.vander(L[indices], m, increasing=True)*weights
    alpha= np.multiply(np.conj(R.T)@R, np.conj(pom @ np.conj(pom.T)))
    G = (np.conj(Q.T) @ X)[:l, :]  
    alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G))@np.ones((m, 1)), assume_a='pos')
    #alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G*weights))@np.ones((m, 1)), assume_a='pos')
    return Z, L[indices], alpha.reshape(-1)

def DMD_reconstruction(X, Z, indices, L, times, weights=None, real=True): #mozda da prima vektor napraviti..
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S,
    Z = dmd modes (returned from some version of DMD, ex. GEDMDQ)
    L = dmd eigs (returned from some version of DMD, ex. GEDMDQ)
    time = integer, which datasnapshot you want to reconstruct/predict"""
    Z_l, L, alpha = DMD_alpha_for_reconstruction(X, Z, indices, L, weights)
    num=np.asarray(times).shape[0]
    recs = np.empty((Z_l.shape[0], num), dtype='complex_')
    for i in range(num):
        recs[:,i] = Z_l@(L**(times[i])*alpha) #mislim da je times[i] jer ovdje krecemo od 0, za razliku od matlaba gdje krecemo od 1
    if real:
        return np.real(recs) #ako sve realno, ovo ce biti realno za svaki i, samo ce zapis biti u obliku kompleksnog; zato saljemo np.real(recs)!
    else:
        return recs
